# Lab Five: Wide and Deep Networks


Arely Alcantara, Emily Fashenpour

## 1. Preparation and Overview

### 1.1 Business Case

The data set we selected is titled "Stack Overflow 2018 Developer Survey". It is a yearly survey collected by Stack Overflow where the developer community is asked questions that range from education, job satisfaction, views on AI, adverstising, ethics, and even stack overflow itself, and general sleeping and eating habits. There were more than 100,000 responses to the survey but there were only 67,441 responses that were completed and did not contain personally identifying information. 

There are two .csv files that below to this dataset. The first file (survey_results_public.csv) contains all the responses to the questions asked in the survey and the second file (survey_results_schema.csv) contains the question the goes with each column in the first file. For example, the column 'Hobby' contains all the responses to the question 'Do you code as a hobby?'.

There are several job seach companies, like Indeed and ZipRecruiter, who could use data from a survey like this to help find more customers. Meaning they could take this data, try to determine which types of features correlate to a developer currently looking for a job, and better understand who they should be connecting with to say 'Hey, we think you might be looking for a job, and here are some jobs that you may be interested in!'.

We believe that in order for companies like Indeed to find our model useful, we need to be precise more than half of the time. That just means that our model needs to be able to predict and be right more than half of time so that it would be better than just guessing.

* Dataset URL: https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey#survey_results_public.csv
* Classification task: Classify a developer's job search status as either not interested in new job opportunites, not actively searching but open to hearing about job opportunites, or actively searching for new job opportunities.

### 1.2 Data Preparation

There are a total of 129 features in the dataset. To help with time and complexity of building the models to classify whether a developer is currently searching for a job, we decided to drop a lot of the features for a several reasons. (1) There were a couple columns that we felt would have no affect on the model and the prediction task including 'Respondent', which was just a unique number for each developer who completed the survey and 'IDE' which contained all the responses to which IDEs the developer used. (2) There were a lot of questions that asked the developer to rank certain values in order of importance, like prefered way to be contacted at work, how they felt about a particular advertisment, or how they thought stack overflow could improve. We felt that these features would not have an affect on the model/prediction task. (3) __HERE__

What is left after dropping many of the columns are the features we felt would have an impact on the prediction task. Features like 'JobSatisfaction', 'Age', and 'Dependents'. If developers are unhappy with their current job, are they more likely to be looking for a job and do the number of dependents and age also affect whether they are looking for a new job?

Since we are trying to predict whether a developer is currently searching for a job, we are also going to drop all the rows in 'JobSearchStatus' that have a null value or 'nan'.

In [163]:
import pandas as pd
import numpy as np


#read in the csv files
survey = pd.read_csv('stack-overflow-survey/survey_results_public.csv')

#drop colums that are not needed
drop = ['Respondent', 'SurveyEasy', 'SurveyTooLong', 'AssessJob1', 'AssessJob2', 'AssessJob3',
        'AssessJob4', 'AssessJob5', 'AssessJob6', 'AssessJob7', 'AssessJob8', 'AssessJob9', 'AssessJob10',
        'AssessBenefits1', 'AssessBenefits2', 'AssessBenefits3', 'AssessBenefits4', 'AssessBenefits5',
        'AssessBenefits6', 'AssessBenefits7', 'AssessBenefits8', 'AssessBenefits9', 'AssessBenefits10',
        'AssessBenefits11', 'JobContactPriorities1', 'JobContactPriorities2', 'JobContactPriorities3', 
        'JobContactPriorities4', 'JobContactPriorities5', 'JobEmailPriorities1', 'JobEmailPriorities2',
        'JobEmailPriorities3', 'JobEmailPriorities4', 'JobEmailPriorities5', 'JobEmailPriorities6',
        'JobEmailPriorities7', 'Currency', 'Salary', 'CurrencySymbol', 'IDE', 'OperatingSystem', 'NumberMonitors',
        'AdBlocker', 'AdBlockerDisable', 'AdBlockerReasons', 'AdsAgreeDisagree1', 'AdsAgreeDisagree2', 
        'AdsAgreeDisagree3', 'AdsActions', 'AdsPriorities1', 'AdsPriorities2', 'AdsPriorities3', 
        'AdsPriorities4', 'AdsPriorities5', 'AdsPriorities6','AdsPriorities7', 'AIDangerous', 
        'AIInteresting', 'AIResponsible', 'AIFuture', 'EthicsChoice', 'EthicsReport','EthicsResponsible',
        'EthicalImplications', 'StackOverflowRecommend', 'StackOverflowVisit', 'StackOverflowHasAccount', 
        'StackOverflowParticipate', 'StackOverflowDevStory', 'StackOverflowJobsRecommend', 'StackOverflowConsiderMember',
        'HypotheticalTools1', 'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'HypotheticalTools5', 
        'ErgonomicDevices', 'LanguageWorkedWith' , 'LanguageDesireNextYear', 'DatabaseWorkedWith',
        'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 'FrameworkWorkedWith', 
        'FrameworkDesireNextYear', 'Methodology', 'VersionControl', 'CommunicationTools', 'TimeFullyProductive',
        'SelfTaughtTypes', 'TimeAfterBootcamp', 'CheckInCode']# ['SurveyEasy', 'SurveyTooLong']

#dropping all col
for d in drop:
    survey.drop([d], axis=1, inplace=True)

#dropping all null values in the 'JobSearchStaus' column
survey = survey.dropna(subset=['JobSearchStatus', 'CareerSatisfaction', 'Employment', 'JobSatisfaction', 'HopeFiveYears', 'LastNewJob', 'UpdateCV'])

survey.head()

,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,...,HoursOutside,SkipMeals,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS
0,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,3-5 years,...,1 - 2 hours,Never,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN
1,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, physics)","10,000 or more employees",Database administrator;DevOps specialist;Full-stack developer;System administrator,30 or more years,...,30 - 59 minutes,Never,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN
3,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or software engineering",100 to 499 employees,Full-stack developer,18-20 years,...,Less than 30 minutes,3 - 4 times per week,I don't typically exercise,Male,Straight or heterosexual,Some college/university study without earning a degree,White or of European descent,35 - 44 years old,No,No
4,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning a degree,"Computer science, computer engineering, or software engineering","10,000 or more employees",Data or business analyst;Desktop or enterprise applications developer;Game or graphics developer;QA or test developer;Student,6-8 years,...,1 - 2 hours,Never,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning a degree,White or of European descent,18 - 24 years old,Yes,NaN
5,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or software engineering",10 to 19 employees,Back-end developer;Database administrator;Front-end developer;Full-stack developer,6-8 years,...,30 - 59 minutes,1 - 2 times per week,1 - 2 times per week,Male,Straight or heterosexual,"Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",White or of European descent,18 - 24 years old,No,NaN


In [164]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58236 entries, 0 to 80449
Data columns (total 37 columns):
Hobby                 58236 non-null object
OpenSource            58236 non-null object
Country               58236 non-null object
Student               57635 non-null object
Employment            58236 non-null object
FormalEducation       57423 non-null object
UndergradMajor        52307 non-null object
CompanySize           51038 non-null object
DevType               57914 non-null object
YearsCoding           58210 non-null object
YearsCodingProf       58236 non-null object
JobSatisfaction       58236 non-null object
CareerSatisfaction    58236 non-null object
HopeFiveYears         58236 non-null object
JobSearchStatus       58236 non-null object
LastNewJob            58236 non-null object
UpdateCV              58236 non-null object
SalaryType            44441 non-null object
ConvertedSalary       43346 non-null float64
EducationTypes        54844 non-null object
HackathonR

Below is a table of what each column in our dataset means - in terms of the questions that the survey takers were asked. We merged both csv files and this is just to show what each attribute is as the column names might not be too descriptive.

In [165]:
data_des = pd.DataFrame()
data_des['Column'] = survey.columns
survey_schema = pd.read_csv('stack-overflow-survey/survey_results_schema.csv')
data_des = pd.merge(data_des, survey_schema, on=['Column'], how="inner", indicator=False)
pd.options.display.max_colwidth = 150
data_des

,Column,QuestionText
0,Hobby,Do you code as a hobby?
1,OpenSource,Do you contribute to open source projects?
2,Country,In which country do you currently reside?
3,Student,"Are you currently enrolled in a formal, degree-granting college or university program?"
4,Employment,Which of the following best describes your current employment status?
5,FormalEducation,Which of the following best describes the highest level of formal education that you’ve completed?
6,UndergradMajor,You previously indicated that you went to a college or university. Which of the following best describes your main field of study (aka 'major')
7,CompanySize,Approximately how many people are employed by the company or organization you work for?
8,DevType,Which of the following describe you? Please select all that apply.
9,YearsCoding,"Including any education, for how many years have you been coding?"


The majority of columns contain categorical data that need to be mapped to an interger value. For example, the 'JobSearchStatus' has three unique values: 'I am not interested in new job opportunities','I’m not actively looking, but I am open to new opportunities', and 'I am actively looking for a job' which are mapped in integer values 0, 1, and 2. The same process if repeated for the other columns.

In [166]:
#map categorical data to integer values
survey['JobSearchStatus'] = survey['JobSearchStatus'].map({
    'I am not interested in new job opportunities': 0,
    'I’m not actively looking, but I am open to new opportunities': 1,
    'I am actively looking for a job': 2
})

A = pd.Series(survey['JobSearchStatus']).unique()
for a in A:
    print('Unique:  ',a)

Unique:   1
Unique:   2
Unique:   0


We have a lot of categorical features that have varying degrees of agree/disagree, yes or no questions, as well as empty responses so we did our best to one hot encode each categorical variable and replace NaN's with a neutral value.

In [167]:
# #map categorical data to integer values
survey['Hobby'] = survey['Hobby'].map({
    'No': 0,
    'Yes': 1
})

survey['OpenSource'] = survey['OpenSource'].map({
    'No': 0,
    'Yes': 1
})

survey['Student'] = survey['Student'].map({
    'No': 0,
    'Yes, part-time': 1,
    'Yes, full-time': 2
})
survey['Student'] = survey['Student'].fillna(0)
survey['Student'] = survey['Student'].astype(int)

survey['Employment'] = survey['Employment'].map({
    'Independent contractor, freelancer, or self-employed': 0,
    'Employed part-time': 1,
    'Employed part-time': 2
})

survey['JobSatisfaction'] = survey['JobSatisfaction'].map({
    'Extremely dissatisfied': 0,
    'Moderately dissatisfied': 1,
    'Slightly dissatisfied': 2,
    'Neither satisfied nor dissatisfied': 3,
    'Slightly satisfied': 4,
    'Moderately satisfied': 5,
    'Extremely satisfied': 6
})
survey['JobSatisfaction'] = survey['JobSatisfaction'].astype(int)

survey['CareerSatisfaction'] = survey['CareerSatisfaction'].map({
    'Extremely dissatisfied': 0,
    'Moderately dissatisfied': 1,
    'Slightly dissatisfied': 2,
    'Neither satisfied nor dissatisfied': 3,
    'Slightly satisfied': 4,
    'Moderately satisfied': 5,
    'Extremely satisfied': 6
})
survey['CareerSatisfaction'] = survey['CareerSatisfaction'].astype(int)

survey['HopeFiveYears'] = survey['HopeFiveYears'].map({
    "Working as a founder or co-founder of my own company": 0,
    "Working in a different or more specialized technical role than the one I'm in now": 1,
    "Doing the same work": 2,
    "Working in a career completely unrelated to software development": 3,
    "Working as a product manager or project manager": 4,
    "Working as an engineering manager or other functional manager": 5,
    "Retirement": 6
})
survey['HopeFiveYears'] = survey['HopeFiveYears'].astype(int)

survey['LastNewJob'] = survey['LastNewJob'].map({
    "I've never had a job": 0,
    "Less than a year ago": 1,
    "Between 1 and 2 years ago": 2,
    "Between 2 and 4 years ago": 3,
    "More than 4 years ago": 4
})
survey['LastNewJob'] = survey['LastNewJob'].astype(int)

survey['UpdateCV'] = survey['UpdateCV'].map({
    "My job status or other personal status changed": 0,
    "I saw an employer’s advertisement": 1,
    "A recruiter contacted me": 2,
    "I did not receive an expected change in compensation": 3,
    "A friend told me about a job opportunity": 4,
    "I had a negative experience or interaction at work": 5,
    "I received bad news about the future of my company or department": 6,
    "I received negative feedback on my job performance": 7
})
survey['UpdateCV'] = survey['UpdateCV'].astype(int)

survey['AgreeDisagree1'] = survey['AgreeDisagree1'].map({
    'Neither Agree nor Disagree': 0,
    'Strongly disagree': 1,
    'Disagree': 2,
    'Agree': 3,
    'Strongly agree': 4
})
survey['AgreeDisagree1'] = survey['AgreeDisagree1'].fillna(0)
survey['AgreeDisagree1'] = survey['AgreeDisagree1'].astype(int)

survey['AgreeDisagree2'] = survey['AgreeDisagree2'].map({
    'Neither Agree nor Disagree': 0,
    'Strongly disagree': 1,
    'Disagree': 2,
    'Agree': 3,
    'Strongly agree': 4
})
survey['AgreeDisagree2'] = survey['AgreeDisagree2'].fillna(0)
survey['AgreeDisagree2'] = survey['AgreeDisagree2'].astype(int)

survey['AgreeDisagree3'] = survey['AgreeDisagree3'].map({
    'Neither Agree nor Disagree': 0,
    'Strongly disagree': 1,
    'Disagree': 2,
    'Agree': 3,
    'Strongly agree': 4
})
survey['AgreeDisagree3'] = survey['AgreeDisagree3'].fillna(0)
survey['AgreeDisagree3'] = survey['AgreeDisagree3'].astype(int)

survey['StackOverflowJobs'] = survey['StackOverflowJobs'].map({
    "No, I didn't know that Stack Overflow had a jobs board": 0,
    'No, I knew that Stack Overflow had a jobs board but have never used or visited it': 1,
    'Yes': 2
})
survey['StackOverflowJobs'] = survey['StackOverflowJobs'].fillna(0)
survey['StackOverflowJobs'] = survey['StackOverflowJobs'].astype(int)

survey['SkipMeals'] = survey['SkipMeals'].map({
    'Never': 0,
    '1 - 2 times per week': 1,
    '3 - 4 times per week': 2,
    'Daily or almost every day': 3
})
survey['SkipMeals'] = survey['SkipMeals'].fillna(0)
survey['SkipMeals'] = survey['SkipMeals'].astype(int)

survey['Exercise'] = survey['Exercise'].map({
    'Never': 0,
    '1 - 2 times per week': 1,
    '3 - 4 times per week': 2,
    'Daily or almost every day': 3
})
survey['Exercise'] = survey['Exercise'].fillna(0)
survey['Exercise'] = survey['Exercise'].astype(int)

survey['EducationParents'] = survey['EducationParents'].map({
    'They never completed any formal education': 0,
    'Primary/elementary school': 1,
    'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 2,
    'Some college/university study without earning a degree': 3,
    'Associate degree': 4,
    'Bachelor’s degree (BA, BS, B.Eng., etc.)': 5,
    'Master’s degree (MA, MS, M.Eng., MBA, etc.)': 6,
    'Professional degree (JD, MD, etc.)': 7,
    'Other doctoral degree (Ph.D, Ed.D., etc.)': 8
})
survey['EducationParents'] = survey['EducationParents'].fillna(0)
survey['EducationParents'] = survey['EducationParents'].astype(int)

survey['Dependents'] = survey['Dependents'].map({
    'No': 0,
    'Yes': 1
})
survey['Dependents'] = survey['Dependents'].fillna(0)
survey['Dependents'] = survey['Dependents'].astype(int)

survey['MilitaryUS'] = survey['MilitaryUS'].map({
    'No': 0,
    'Yes': 1
})
survey['MilitaryUS'] = survey['MilitaryUS'].fillna(0)
survey['MilitaryUS'] = survey['MilitaryUS'].astype(int)

In [170]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58236 entries, 0 to 80449
Data columns (total 37 columns):
Hobby                 58236 non-null int64
OpenSource            58236 non-null int64
Country               58236 non-null object
Student               58236 non-null int64
Employment            9165 non-null float64
FormalEducation       57423 non-null object
UndergradMajor        52307 non-null object
CompanySize           51038 non-null object
DevType               57914 non-null object
YearsCoding           58210 non-null object
YearsCodingProf       58236 non-null object
JobSatisfaction       58236 non-null int64
CareerSatisfaction    58236 non-null int64
HopeFiveYears         58236 non-null int64
JobSearchStatus       58236 non-null int64
LastNewJob            58236 non-null int64
UpdateCV              58236 non-null int64
SalaryType            44441 non-null object
ConvertedSalary       43346 non-null float64
EducationTypes        54844 non-null object
HackathonReasons   

### 1.3 Cross Product Features

In [178]:
crossed_colums = [['Student', 'Country'], 
                  ['Hobby', 'Employment'], 
                  ['JobSatisfaction', 'SkipMeals', 'AgreeDisagree1'],
                  ['CareerSatisfaction', 'HopeFiveYears', 'UpdateCV', 'AgreeDisagree3', 'JobSatisfaction']]

We will be creating groups of crossed columns in our data so that our model is able to memorize and pick up on certain patterns. We hope to train our model with these groups and see if there's any relationships that cause someone to actively search for a job and what those motivations might be. We understand that there might be outside factors that could also affect someone's desire for a new job, but we feel that the stackoverflow survey really captured the essence of a person and we picked the questions that are most relevant to us. As far as the groups go, we decided to look at Student and Country - so will a person being in school in Europe or any country in that matter, be interested in a job? We will also look at Hobby and Employment - so, does an individual code for fun and are they currently employed? The next group is JobSatisfaction, SkipMeals, AgreeDisagree1 - which deals with how satisfied or unsatisfied an individual is with their current job, how many meals they skip a week, and how well they get along with their peers at work (overall, reasons that could cause one to want to leave their job and search for a new one.). And lastly, we look at how satisfied an individual is with their career, where they see themselves in 5 years, if they have updated their resume recently, how they feel in relation to their peers in terms of efficiency, and how satisfied they are at work. We feel that by having these groups, we will have a better understanding of what could cause someone to want to look for a new job, or not want to get a new job since they're employed.

### 1.4 Evaluation Metrics

Jobs are extremely important and we want to help people get connected with companies if they're looking for a job, and also not reach out or promote jobs with people who are employed and not interested in a new job.. because that would be awkward. Our goal is to be able to predict whether a person is looking for a job, or not, or maybe just checking out what's out there - whatever it is, we want to help. In order to do that, we need to make sure that our model is precise - meaning that we're able to get close values, being consistent is key for us. But we also want to be accurate in predicting so getting the correct predictions each time. We will be using both accuracy and precision to measure the performance of our model in predicting a person's job search status.

### 1.5 Dividing Data into Training and Testing

Since we have such a big dataset, we have decided to stick with an 80/20 split. We will use 80% of the dataset to train our model and use the remaining 20% to test how well our model performs. Using any types of folding or shuffling would be terribly slow and almost unnecessary in our case, wo we will just split the dataset to get an idea of what job hunters look like versus full time students or happily employed people.

In [180]:
from sklearn.model_selection import train_test_split

# split dataframe into train and test sets
train, test = train_test_split(survey,test_size=0.2)

## 2. Modeling

### 2.1 Model 1

### 2.2 Model 2

### 2.3 Model 3

### 2.4 Comparing Our Best Model to a Standard MultiLayer Perceptron

#### ROC Graph

## 3. Additional Analysis

### 3.1 Dimensionality Reduction and Visualization

## References
https://www.kaggle.com/stackoverflow/stack-overflow-2018-developer-survey#survey_results_public.csv